In [66]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,root_mean_squared_error,r2_score

In [14]:
df = pd.read_csv('car-details.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6926 entries, 0 to 6925
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6926 non-null   object 
 1   company        6926 non-null   object 
 2   model          6926 non-null   object 
 3   edition        6926 non-null   object 
 4   year           6926 non-null   int64  
 5   owner          6926 non-null   object 
 6   fuel           6926 non-null   object 
 7   seller_type    6926 non-null   object 
 8   transmission   6926 non-null   object 
 9   km_driven      6926 non-null   int64  
 10  mileage_mpg    6718 non-null   float64
 11  engine_cc      6718 non-null   float64
 12  max_power_bhp  6717 non-null   float64
 13  torque_nm      6717 non-null   float64
 14  seats          6718 non-null   float64
 15  selling_price  6926 non-null   int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 865.9+ KB


In [16]:
df.isna().sum()

name               0
company            0
model              0
edition            0
year               0
owner              0
fuel               0
seller_type        0
transmission       0
km_driven          0
mileage_mpg      208
engine_cc        208
max_power_bhp    209
torque_nm        209
seats            208
selling_price      0
dtype: int64

In [18]:
df.shape

(6926, 16)

In [23]:
for col in df.select_dtypes(include='O').columns:
    print(f"Column: {col}")
    print(f"Cardinality:{df[col].nunique()}")
    print(df[col].unique())
    print(df[col].value_counts(normalize=True))
    print()
          

Column: name
Cardinality:2058
['Maruti Swift Dzire VDI' 'Skoda Rapid 1.5 TDI Ambition'
 'Honda City 2017-2020 EXi' ... 'Tata Nexon 1.5 Revotorq XT'
 'Ford Freestyle Titanium Plus Diesel BSIV'
 'Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV']
name
Maruti Swift Dzire VDI                       0.017037
Maruti Alto 800 LXI                          0.010973
Maruti Alto LXi                              0.009962
Maruti Swift VDI                             0.008663
Maruti Swift VDI BSIV                        0.008085
                                               ...   
Tata Tiago 1.2 Revotron XZ Plus Dual Tone    0.000144
Mahindra KUV 100 mFALCON G80 K4 Plus 5str    0.000144
Hyundai Verna SX Diesel                      0.000144
Maruti S-Presso VXI AT                       0.000144
Hyundai Santro Xing XK eRLX EuroIII          0.000144
Name: proportion, Length: 2058, dtype: float64

Column: company
Cardinality:32
['Maruti' 'Skoda' 'Honda' 'Hyundai' 'Toyota' 'Ford' 'Renault' 'Mahindra'
 'Tata' 

In [24]:
df = df.drop(columns=['name','model','edition'])


In [25]:
df.duplicated().sum()

np.int64(19)

In [26]:
df.drop_duplicates(inplace=True)

In [28]:
X = df.drop(columns='selling_price')
y = df.selling_price.copy()

In [31]:
X

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00,1248.0,74.00,190.000000,5.0
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70,1498.0,103.52,250.000000,5.0
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60,1497.0,78.00,124.544455,5.0
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06,1396.0,90.00,219.668960,5.0
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84,1298.0,88.20,112.776475,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6921,Maruti,2013,Second,Petrol,Individual,Manual,50000,44.40,998.0,67.10,90.000000,5.0
6922,Hyundai,2014,Second,Diesel,Individual,Manual,80000,52.97,1396.0,88.73,219.700000,5.0
6923,Hyundai,2013,First,Petrol,Individual,Manual,110000,43.47,1197.0,82.85,113.700000,5.0
6924,Hyundai,2007,Fourth & Above,Diesel,Individual,Manual,119000,39.47,1493.0,110.00,235.359600,5.0


In [34]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(5525, 12) (5525,)
(1382, 12) (1382,)


In [40]:
num_col = X_train.select_dtypes(include='number').columns.tolist()
cat_col = [col for col in X_train.columns if col not in num_col]
print(num_col)
print(cat_col)

['year', 'km_driven', 'mileage_mpg', 'engine_cc', 'max_power_bhp', 'torque_nm', 'seats']
['company', 'owner', 'fuel', 'seller_type', 'transmission']


In [45]:
num_pipe = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
        ])
cat_pipe = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='constant',fill_value='missing')),
        ('encoder',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
    ]
)

preprocessor = ColumnTransformer(transformers=[
    ('num',num_pipe,num_col),\
    ('cat',cat_pipe,cat_col)
])

In [46]:
X_train

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats
3442,Mahindra,2012,Second,Diesel,Individual,Manual,120000,28.31,2179.0,120.00,290.000000,8.0
1883,Tata,2011,First,Diesel,Individual,Manual,156000,27.19,2179.0,138.10,320.000000,7.0
5200,Mahindra,2012,First,Diesel,Individual,Manual,120000,37.50,2523.0,62.10,195.000000,7.0
969,Ford,2011,Third,Diesel,Individual,Manual,110000,41.84,1399.0,68.00,159.848395,5.0
4520,Tata,2017,First,Petrol,Individual,Manual,20000,56.03,1199.0,84.00,114.000000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3780,Hyundai,2017,First,Petrol,Individual,Manual,5000,44.40,1197.0,81.86,113.750000,5.0
5204,Tata,2010,Second,Diesel,Individual,Manual,148000,44.88,1396.0,69.00,135.000000,5.0
5240,Honda,2017,First,Petrol,Individual,Manual,40000,40.88,1497.0,117.60,145.000000,5.0
5404,Honda,2013,Second,Petrol,Dealer,Manual,55000,39.94,1497.0,118.00,146.000000,5.0


In [54]:
preprocessor.fit_transform(X_train)

array([[-0.34663648,  0.74073059, -1.58876246, ...,  0.        ,
         0.        ,  1.        ],
       [-0.5920448 ,  1.32832009, -1.69521801, ...,  0.        ,
         0.        ,  1.        ],
       [-0.34663648,  0.74073059, -0.71525667, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.88040513, -0.56502387, -0.39398904, ...,  0.        ,
         0.        ,  1.        ],
       [-0.10122816, -0.32019491, -0.48333566, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.12581345, -0.07536595,  1.0811807 , ...,  0.        ,
         0.        ,  1.        ]], shape=(5525, 51))

In [56]:
regressor = RandomForestRegressor(
    n_estimators=10,max_depth=5,random_state=42
)

In [61]:
rf_model = Pipeline(steps=[
    ('pre',preprocessor),
    ('reg',regressor)
])

In [63]:
rf_model.fit(X_train,y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'km_driven',
                                                   'mileage_mpg', 'engine_cc',
                                                   'max_power_bhp', 'torque_nm',
                                                   'seats']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['company', 'owner', 'fuel',
                                                   'seller_type',
                                                   'transmission'])])),
                ('reg',
                 RandomForestRegressor(max_depth=5, n_estimators=10,
                                       random_state=42))])

In [68]:
y_train_pred = rf_model.predict(X_train)
print(f"RMSE of training data: {root_mean_squared_error(y_train,y_train_pred)}")
print(f"R-Squared of training data: {r2_score(y_train,y_train_pred)}")

RMSE of training data: 169947.48964050272
R-Squared of training data: 0.8979695215795949


In [69]:
y_test_pred = rf_model.predict(X_test)
print(f"RMSE of Testing data: {root_mean_squared_error(y_test,y_test_pred)}")
print(f"R-Squared of Testing data: {r2_score(y_test,y_test_pred)}")

RMSE of Testing data: 172392.1313605195
R-Squared of Testing data: 0.865675993366697
